#CARICAMENTO LIBRERIE E COLLEGAMENTO DRIVE

##Librerie

In [ ]:
!pip install audiomentations[extras] --quiet
!pip install substring --quiet

In [2]:
!PYTHONHASHSEED=0
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot as plt
import zipfile
from shutil import copyfile
from time import time
import sklearn
import numpy as np
import pandas as pd
import random as python_random
import os
np.random.seed(0)
python_random.seed(0)
tf.random.set_seed(0)

from google.colab import drive

import librosa
import librosa.display
import math, random
from matplotlib.ticker import MaxNLocator
from IPython.display import Audio
import torch
import torchaudio
from torchaudio import transforms
from torch.utils.data import DataLoader, Dataset, random_split
from matplotlib import pyplot

import torch.nn.functional as F
import torch.nn as nn
from torch.nn import init

from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, PolarityInversion, Reverse

# Impostazioni di visualizzazione migliorata in Google Colab
from IPython.display import Javascript
from substring import substringByInd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torchvision import transforms as trs

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

def resize_colab_cell():
  display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)

##Importazione dati da Google Drive

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
copyfile('/content/gdrive/MyDrive/Deep/BerlinSpeech.zip', 'BerlinSpeech.zip')

In [ ]:
zip = zipfile.ZipFile('BerlinSpeech.zip')
zip.extractall()
zip.close()

#CARICAMENTO DATASET

##Il nome del file audio contiene informazioni sul contenuto, come ad esempio l'emozione espressa nell'audio (target) 

1. I primi due numeri rappresentano l'identificativo dell'attore
2. La stringa alfanumerica di 3 caratteri in mezzo identifica la frase
3. Le ultime due le iniziali del sentimento, la prima riferita all'inglese e la seconda al tedesco. 


In [ ]:
# Si inizializzano i dizionari con la quale si ricaverà poi il target per ogni audio, a partire dal nome del file.
de_emotions = { 'W': 'anger',
                'L': 'boredom',
                'E': 'disgust',
                'A': 'fear',
                'F': 'happiness',
                'T': 'sadness',
                'N': 'neutral',
                }

mapping = {'anger':0,
           'boredom':1,
           'disgust':2,
           'fear':3,
           'happiness':4,
           'sadness':5,
           'neutral':6,
           }

In [ ]:
df = pd.DataFrame()
audio = []
label = []
actor_label = []
for idx,speech_name in enumerate(os.listdir('/content/wav')):
  audio.append('/wav/' + speech_name)
  genre = de_emotions[speech_name[5]]
  actor = substringByInd(str(speech_name), 0, 1)
  label.append(mapping[genre])
  actor_label.append(actor)


df['audio'] = audio
df['etichetta'] = label
df['attore'] = actor_label
df['etichett-attore'] = df['etichetta'].astype(str) + df['attore'].astype(str)
unique_authors = df['etichett-attore'].value_counts()==1
unique_authors = unique_authors[unique_authors].index
for index in unique_authors:
  df['etichett-attore'][df['etichett-attore']==index]='999'
df

#FUNZIONI PER IL PREPROCESSING E PER L'ALLENAMENTO DELLA RETE

## Utility per la gestione delle tracce audio

In [ ]:
class AudioUtil():

  # ----------------------------
  # Load an audio file. Return the signal as a tensor and the sample rate
  # ----------------------------
  @staticmethod
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    return (sig, sr)



  # ----------------------------
  # Convert the given audio to the desired number of channels
  # ----------------------------
  @staticmethod
  def rechannel(aud, new_channel):
    sig, sr = aud

    if (sig.shape[0] == new_channel):
        # Nothing to do
      return aud

    if (new_channel == 1):
        # Convert from stereo to mono by selecting only the first channel
      resig = sig[:1, :]
    else:
        # Convert from mono to stereo by duplicating the first channel
      resig = torch.cat([sig, sig])

    return ((resig, sr))


  # ----------------------------
  # Since Resample applies to a single channel, we resample one channel at a time
  # ----------------------------
  @staticmethod
  def resample(aud, newsr):
    sig, sr = aud

    if (sr == newsr):
        # Nothing to do
      return aud

    num_channels = sig.shape[0]
      # Resample first channel
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
        # Resample the second channel and merge both channels
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))


  # ----------------------------
  # Pad (or truncate) the signal to a fixed length 'max_ms' in milliseconds
  # ----------------------------
  @staticmethod
  def pad_trunc(aud, max_ms):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms

    if (sig_len > max_len):
        # Truncate the signal to the given length
      sig = sig[:,:max_len]

    elif (sig_len < max_len):
        # Length of padding to add at the beginning and end of the signal
      pad_begin_len = random.randint(0, max_len - sig_len)
      pad_end_len = max_len - sig_len - pad_begin_len

        # Pad with 0s
      pad_begin = torch.zeros((num_rows, pad_begin_len))
      pad_end = torch.zeros((num_rows, pad_end_len))

      sig = torch.cat((pad_begin, sig, pad_end), 1)
        
    return (sig, sr)



   # ----------------------------
  # Shifts the signal to the left or right by some percent. Values at the end
  # are 'wrapped around' to the start of the transformed signal.
  # ----------------------------
  @staticmethod
  def time_shift(aud, shift_limit):
    sig,sr = aud
    _, sig_len = sig.shape
    shift_amt = int(random.random() * shift_limit * sig_len)
    return (sig.roll(shift_amt), sr)


  # ----------------------------
  # Generate a Spectrogram
  # ----------------------------
  @staticmethod  
  def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80

      # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

      # Convert to decibels
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return spec

## Funzione che implementa l'augmentation 

In [ ]:
def augmented_train(df):

  lista = [AddGaussianNoise(p=1, min_amplitude=0.01, max_amplitude=0.02),
            TimeStretch(p=1, min_rate=0.8, max_rate=0.9),
            TimeStretch(p=1, min_rate=1.125, max_rate=1.25),
            PitchShift(p=1, min_semitones=1, max_semitones=3),
            PitchShift(p=1, min_semitones=-3, max_semitones=-1),
            Shift(p=1, fade=True),  #fade_duration=0.01
            PolarityInversion(p=1),
            Reverse(p=1)]

  df_block = pd.DataFrame([], columns=['audio', 'etichetta'])

  for idx,row in enumerate(df.iterrows()):
    audio_file = '/content' + row[1][0]
    aud = AudioUtil.open(audio_file)
    class_id = row[1][1]
    

    reaud = AudioUtil.resample(aud, 22050)
    rechan = AudioUtil.rechannel(reaud, 1)
    dur_aud = AudioUtil.pad_trunc(rechan, 4000)
    torchaudio.save(audio_file,dur_aud[0], 22050)

    functions = list(np.random.choice(a = lista, replace = False, size = 3, p=[1/6,1/12,1/12,1/12,1/12,1/6,1/6,1/6]))
   
    for index, func in enumerate(functions):
      augment = Compose([func])
      if (func != lista[1]) & (func != lista[2]):     
        augmented_samples = augment(samples = dur_aud[0], sample_rate=22050)
      else:
        dur_aud_1 = np.asarray(dur_aud[0])                   # Time Stretch vuole in input un ndarray e non un tensore...
        augmented_samples = augment(samples = dur_aud_1, sample_rate=22050)
        augmented_samples = torch.from_numpy(augmented_samples)

      if torch.is_tensor(augmented_samples)== False:
        augmented_samples = torch.from_numpy(augmented_samples.copy())
      augmented_samples
      augmented_filename = '/content'  + substringByInd(str(row[1][0]), 0, 11) + '_aug' + str(index) + '.wav' 
      torchaudio.save(augmented_filename,augmented_samples,22050)

In [ ]:
augmented_train(df)

## Funzioni che implementano gli spettrogrammi

In [ ]:
def compute_spectrogram(df):

  df_block = pd.DataFrame([], columns=['audio', 'etichetta'])

  for idx,row in enumerate(df.iterrows()):
    audio_file = '/content' + row[1][0]
    aud = AudioUtil.open(audio_file)
    class_id = row[1][1]    
    spectrogram = AudioUtil.spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None)
    data = {'audio': spectrogram, 'etichetta': class_id}
    df_block = df_block.append(data, ignore_index = True)
    for index in range(3):
      audio_file =  '/content'  + substringByInd(str(row[1][0]), 0, 11) + '_aug' + str(index) + '.wav'
      aud = AudioUtil.open(audio_file)
      spectrogram = AudioUtil.spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None)
      data = {'audio': spectrogram, 'etichetta': class_id}
      df_block = df_block.append(data, ignore_index = True)
  return df_block

In [ ]:
def compute_spectrogram_validation(df):

  df_block = pd.DataFrame([], columns=['audio', 'etichetta'])

  for idx,row in enumerate(df.iterrows()):
    audio_file = '/content' + row[1][0]
    aud = AudioUtil.open(audio_file)
    class_id = row[1][1]    
    spectrogram = AudioUtil.spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None)
    data = {'audio': spectrogram, 'etichetta': class_id}
    df_block = df_block.append(data, ignore_index = True)
    
  return df_block

## Funzione che implementa l'inferenza del modello sul validation

In [ ]:
# ----------------------------
# Inference
# ----------------------------

def inference (dataset, model, criterion, val_dl, batch__size):
  correct_prediction = 0
  total_prediction = 0
  running_loss_val = 0
  acc_diz = {0:[0,0,0],1:[0,0,0],2:[0,0,0],3:[0,0,0],4:[0,0,0],5:[0,0,0],6:[0,0,0]}
 
  # Disable gradient updates
  with torch.no_grad():
    for data in val_dl:
      # Get the input features and target labels, and put them on the GPU

      test_spec = compute_spectrogram_validation(dataset.iloc[data.tolist()])
      test_sound = SoundDS(test_spec)
      test_tensor = torch.utils.data.DataLoader(test_sound, batch_size=batch__size)
      for i in test_tensor:
        tt = i
        # Get the input features and target labels, and put them on the GPU
      inputs, labels = tt[0].to(device), tt[1].to(device)

      # Normalize the inputs
      inputs_m, inputs_s = inputs.mean(), inputs.std()
      inputs = (inputs - inputs_m) / inputs_s


      # Get predictions
      
      outputs = model(inputs)
      val_loss = criterion(outputs, labels).item()
      running_loss_val += val_loss

      # Get the predicted class with the highest score
      _, prediction = torch.max(outputs,1)
      
      # Count of predictions that matched the target label
      correct_prediction += (prediction == labels).sum().item()
      total_prediction += prediction.shape[0]

      for label_index,label in enumerate(labels):
          acc_diz[int(label)][0] += 1 
          acc_diz[int(label)][1] += int(label == prediction[label_index])
          acc_diz[int(prediction[label_index])][2] += 1      
  num_batches_val = len(val_dl)
  avg_loss_val = running_loss_val / num_batches_val
  acc = correct_prediction/total_prediction
  print(f'Validation Accuracy: {acc:.2f}, Validation Loss: {avg_loss_val:.2f},Total items: {total_prediction}')
  return acc, avg_loss_val, acc_diz

## Funzioni che valutano l'andamento dell'accuracy e della loss di train e validation durante le epoche di addestramento

In [ ]:
def visualize_acc(training, validation): 

# Sintesi risultati K-fold 
    if isinstance(training[0], list):
      # Taglio al numero di epoche minimo nelle 3 fold (Early Stopping)
      training = [training[j][0:(min([len(training[i]) for i in range(len(training))]))] for j in range(len(training))]
      validation = [validation[j][0:(min([len(validation[i]) for i in range(len(validation))]))] for j in range(len(validation))]
      acc = np.mean(np.array(training), axis = 0)
      val_acc = np.mean(np.array(validation), axis = 0)
# Modello iniziale senza K-fold
    else:
      acc = training
      val_acc = validation
    epochs = range(len(acc))
    figure = plt.figure()
    plt.style.use('dark_background')
    plt.plot(epochs, acc, "b", label="Accuratezza Training")
    plt.plot(epochs, val_acc, "r", label="Accuratezza Validation")
    plt.title("Accuratezza Training e Validation")
    plt.xlabel("Epoca",color='white')
    plt.ylabel("Accuratezza",color='white')
    plt.xticks(range(0,len(acc),3))
    figure.set_facecolor('black')
    plt.legend()
    plt.show()

def visualize_loss(training, validation):

# Sintesi risultati K-fold  
    if isinstance(training[0], list):
      # Taglio al numero di epoche minimo nelle 3 fold (Early Stopping)
      training = [training[j][0:(min([len(training[i]) for i in range(len(training))]))] for j in range(len(training))]
      validation = [validation[j][0:(min([len(validation[i]) for i in range(len(validation))]))] for j in range(len(validation))]
      loss = np.mean(np.array(training), axis = 0)
      val_loss = np.mean(np.array(validation), axis = 0)
# Modello iniziale senza K-fold
    else:
      loss = training
      val_loss = validation
    epochs = range(len(loss))
    figure = plt.figure()
    figure.set_facecolor('black')
    plt.style.use('dark_background')
    plt.plot(epochs, loss, "b", label="Loss Training")
    plt.plot(epochs, val_loss, "r", label="Loss Validation")
    plt.title("Loss Training e Validation")
    plt.xlabel("Epoca",color='white')
    plt.ylabel("Loss",color='white')
    plt.xticks(range(0,len(loss),3))
    plt.legend()
    plt.show()

In [ ]:
import numpy as np
import seaborn as sns

def show_values(axs, orient="v", space=.01):
    def _single(ax):
        if orient == "v":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() / 2
                _y = p.get_y() + p.get_height() + (p.get_height()*0.01)
                value = '{:.3f}'.format(p.get_height())
                ax.text(_x, _y, value, ha="center") 
        elif orient == "h":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() + float(space)
                _y = p.get_y() + p.get_height() - (p.get_height()*0.50 - 0.1)
                value = '{:.3f}'.format(p.get_width())
                ax.text(_x, _y, value, ha="left")

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _single(ax)
    else:
        _single(axs)

def plot_class_recall(acc_diz):
  acc_list = []
  sentiment_list = ['rabbia','noia','disgusto','paura','felicità','tristezza','neutrale']
  for index_, vals in acc_diz.items():
    acc_list.append(vals[1]/vals[0])
  
  sns.set(font_scale = 1.2, rc={'figure.figsize':(7,5),'axes.facecolor':'black', 'figure.facecolor':'black','axes.linewidth': 1,'ytick.color': 'white','xtick.color': 'white','axes.edgecolor': 'grey','axes.grid': False, 'text.color': 'white', 'axes.spines.right': False, 'axes.spines.top': False, 'patch.linewidth': 0.7})
  bar=sns.barplot(x=acc_list, y=sentiment_list, palette='husl')
  show_values(bar,"h", space=0.01)

In [16]:
def plot_class_precision_and_recall(acc_diz,acc_finale):
  rec_prec_df = pd.DataFrame(columns = ['Sentimento', 'Misura', 'Valore'])
  sentiment_list =['rabbia','noia','disgusto','paura','felicità','tristezza','neutrale']
  for index_, vals in acc_diz.items():
    if vals[2]>0:
      rec = {'Sentimento':sentiment_list[index_], 'Misura':'Recall', 'Valore':(vals[1]/vals[0])}
      prec = {'Sentimento':sentiment_list[index_], 'Misura':'Precision', 'Valore':(vals[1]/vals[2])}
      rec_prec_df = rec_prec_df.append(rec, ignore_index=True)
      rec_prec_df = rec_prec_df.append(prec, ignore_index=True)
    else:
      rec = {'Sentimento':sentiment_list[index_], 'Misura':'Recall', 'Valore':vals[1]/vals[0]}
      prec = {'Sentimento':sentiment_list[index_], 'Misura':'Precision', 'Valore':np.inf}
      rec_prec_df = rec_prec_df.append(rec, ignore_index=True)
      rec_prec_df = rec_prec_df.append(prec, ignore_index=True)
      
  sns.set(font_scale = 1.2, rc={'figure.figsize':(11.5,8),'axes.facecolor':'black','axes.labelcolor':'white', 'figure.facecolor':'black','axes.linewidth': 1,'ytick.color': 'white','xtick.color': 'white','axes.edgecolor': 'grey','axes.grid': False, 'text.color': 'white', 'axes.spines.right': False, 'axes.spines.top': False, 'patch.linewidth': 0.5})
  bar=sns.barplot(x='Valore', y='Sentimento', hue='Misura', data=rec_prec_df, palette='mako')
  plt.legend(bbox_to_anchor=(0.95, 0.7), loc='upper left', borderaxespad=0)
  plt.axvline(x=acc_finale,ymin=0,ymax=1,color='pink')
  show_values(bar,"h", space=0.02)

<IPython.core.display.Javascript object>

## Si inizializza una classe speciale che servirà da input al Data Loader

In [ ]:
class SoundDS(Dataset):
  def __init__(self, df):
    self.df = df         

  def __len__(self):
    return len(self.df)    
     
 
  def __getitem__(self, idx):
    
    audio_file = self.df.loc[idx, 'audio']
    
    class_id = self.df.loc[idx, 'etichetta']

    return audio_file, class_id

#COSTRUZIONE RETI E TRAINING MODELLI

## Modello 1: CNN a 2 layer - Batch 16

In [ ]:
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import init

# ----------------------------
# Audio Classification Model
# ----------------------------
class AudioClassifier0 (nn.Module):
    # ----------------------------
    # Build the model architecture
    # ----------------------------
    def __init__(self):
        super().__init__()
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Xavier Initialization
        self.conv1 = nn.Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2,2)
        self.bn1 = nn.BatchNorm2d(8)
        init.xavier_normal_(self.conv1.weight)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1, self.pool1]

        # Second Convolution Block with Relu and Batch Norm. Use Xavier Initialization
        self.conv2 = nn.Conv2d(8, 32, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2,2)
        self.bn2 = nn.BatchNorm2d(32)
        init.xavier_normal_(self.conv2.weight)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2, self.pool2]

        # Dropout 
        self.dropout = nn.Dropout(0.25)

        # Linear Classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=32, out_features=7)
        self.soft = nn.Softmax(dim=1)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)
    
    def forward(self, x):
        # Run the convolutional blocks
        x = self.conv(x)
        # Adaptive pool and flatten for input to linear layer
        x = self.ap(x)
        x = self.dropout(x)
        x = x.view(x.shape[0], -1)  #flattening
        # Linear layer
        x = self.lin(x)
        x = self.soft(x)

        # Final output
        return x

# Create the model and put it on the GPU if available
myModel0 = AudioClassifier0()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel0 = myModel0.to(device)
# Check that it is on Cuda
next(myModel0.parameters()).device

In [ ]:
# ----------------------------
# Training Loop
# ----------------------------
def training0(model, dataset, num_epochs, batch__size):
  # Loss Function, Optimizer and Scheduler
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01,
                                                steps_per_epoch=int(len(dataset)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')
  for layer in model.children():
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()

  # Random split of 2/3:1/3 between training and validation
  num_items = len(dataset)
  num_train = round(num_items * 2/3)
  num_val = num_items - num_train
  train_ds, val_ds, y_train, y_test = train_test_split(dataset.index, dataset['etichett-attore'],test_size=num_val, train_size=num_train,shuffle = True, random_state=123, stratify=dataset['etichett-attore'])


  # Create training and validation data loaders
  train_dl = torch.utils.data.DataLoader(train_ds, batch_size=int(batch__size/4), shuffle = True)
  val_dl = torch.utils.data.DataLoader(val_ds, batch_size=batch__size, shuffle = True)
  accuracy_train = []
  accuracy_val = []
  loss_train = []
  loss_validation = []
  acc_diz_list = []

  # Repeat for each epoch
  for epoch in range(num_epochs):
    running_loss = 0.0
    running_vloss = 0.0
    correct_prediction = 0
    total_prediction = 0

    # Repeat for each batch in the training set
    for i, data in enumerate(train_dl):

        train_spec = compute_spectrogram(dataset.iloc[data.tolist()])
        train_sound = SoundDS(train_spec)
        train_tensor = torch.utils.data.DataLoader(train_sound, batch_size=batch__size)
        for i in train_tensor:
          tt = i
        # Get the input features and target labels, and put them on the GPU
        inputs, labels = tt[0].to(device), tt[1].to(device)

        # Normalize the inputs
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s

        # Zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Keep stats for Loss and Accuracy
        running_loss += loss.item()

        # Get the predicted class with the highest score (posterior, predicted labels)
        _, prediction = torch.max(outputs,1)
    
        # Count of predictions that matched the target label
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]

      # Print stats at the end of the epoch
    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    
    acc = correct_prediction/total_prediction
    acc_val, loss_val, acc_diz = inference(dataset, model, criterion, val_dl, batch__size)
    
    accuracy_train.append(acc)
    accuracy_val.append(acc_val)
    loss_train.append(avg_loss)
    loss_validation.append(loss_val)
    acc_diz_list.append(acc_diz)

    print('Epoch:', epoch+1, 'Loss:', f'{avg_loss:.2f}', 'Accuracy:', acc)

  return accuracy_train, accuracy_val, loss_train, loss_validation, acc_diz_list

In [ ]:
seed = 245
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

num_epochs= 50
batch__size = 16
accuracy0, accuracy_val0, loss0, loss_val0, acc_diz0 = training0(myModel0, df, num_epochs, batch__size)

In [ ]:
visualize_acc(accuracy0,accuracy_val0)

In [ ]:
visualize_loss(loss0,loss_val0)

In [ ]:
index_min = np.argmin(loss_val0)
index_min
plot_class_precision_and_recall(acc_diz0[index_min],accuracy_val0[index_min])

## Modello 2: CNN a 3 layer - Batch 16

In [ ]:
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import init

# ----------------------------
# Audio Classification Model
# ----------------------------
class AudioClassifier2 (nn.Module):
    # ----------------------------
    # Build the model architecture
    # ----------------------------
    def __init__(self):
        super().__init__()
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Xavier Initialization
        self.conv1 = nn.Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2,2)
        self.bn1 = nn.BatchNorm2d(8)
        init.xavier_normal_(self.conv1.weight)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1, self.pool1]

        # Second Convolution Block
        self.conv2 = nn.Conv2d(8, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2,2)
        self.bn2 = nn.BatchNorm2d(32)
        init.xavier_normal_(self.conv2.weight)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2, self.pool2]

        # Third Convolution Block
        self.conv3 = nn.Conv2d(32, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(128)
        init.xavier_normal_(self.conv3.weight)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        # Dropout 
        self.dropout = nn.Dropout(0.25)

        # Linear Classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=128, out_features=7)
        self.soft = nn.Softmax(dim=1)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)
 
    # ----------------------------
    # Forward pass computations
    # ----------------------------
    def forward(self, x):
        # Run the convolutional blocks
        x = self.conv(x)

        # Adaptive pool and flatten for input to linear layer
        x = self.ap(x)
        x = self.dropout(x)
        x = x.view(x.shape[0], -1)

        # Linear layer
        x = self.lin(x)
        x = self.soft(x)

        # Final output
        return x

# Create the model and put it on the GPU if available
myModel2 = AudioClassifier2()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel2 = myModel2.to(device)
# Check that it is on Cuda
next(myModel2.parameters()).device

In [22]:
# ----------------------------
# Training Loop
# ----------------------------
def training2(model, dataset, num_epochs, batch__size):
  # Loss Function, Optimizer and Scheduler
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.01, weight_decay=0.001)
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01,
                                                steps_per_epoch=int(len(dataset)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')
  for layer in model.children():
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()

  # Random split of 2/3:1/3 between training and validation
  num_items = len(dataset)
  num_train = round(num_items * 2/3)
  num_val = num_items - num_train
  train_ds, val_ds, y_train, y_test = train_test_split(dataset.index, dataset['etichett-attore'],test_size=num_val, train_size=num_train,shuffle = True, random_state=123, stratify=dataset['etichett-attore'])


  # Create training and validation data loaders
  train_dl = torch.utils.data.DataLoader(train_ds, batch_size=int(batch__size/4), shuffle = True)
  val_dl = torch.utils.data.DataLoader(val_ds, batch_size=batch__size, shuffle = True)
  accuracy_train = []
  accuracy_val = []
  loss_train = []
  loss_validation = []
  acc_diz_list = []

  # Repeat for each epoch
  for epoch in range(num_epochs):
    running_loss = 0.0
    running_vloss = 0.0
    correct_prediction = 0
    total_prediction = 0

    # Repeat for each batch in the training set
    for i, data in enumerate(train_dl):

        train_spec = compute_spectrogram(dataset.iloc[data.tolist()])
        train_sound = SoundDS(train_spec)
        train_tensor = torch.utils.data.DataLoader(train_sound, batch_size=batch__size)
        for i in train_tensor:
          tt = i
        # Get the input features and target labels, and put them on the GPU
        inputs, labels = tt[0].to(device), tt[1].to(device)

        # Normalize the inputs
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s

        # Zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Keep stats for Loss and Accuracy
        running_loss += loss.item()

        # Get the predicted class with the highest score (posterior, predicted labels)
        _, prediction = torch.max(outputs,1)
    
        # Count of predictions that matched the target label
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]

      # Print stats at the end of the epoch
    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    
    acc = correct_prediction/total_prediction
    acc_val, loss_val, acc_diz = inference(dataset, model, criterion, val_dl, batch__size)
    
    accuracy_train.append(acc)
    accuracy_val.append(acc_val)
    loss_train.append(avg_loss)
    loss_validation.append(loss_val)
    acc_diz_list.append(acc_diz)

    print('Epoch:', epoch+1, 'Loss:', f'{avg_loss:.2f}', 'Accuracy:', acc)

  return accuracy_train, accuracy_val, loss_train, loss_validation, acc_diz_list

<IPython.core.display.Javascript object>

In [ ]:
seed = 784
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

batch__size = 16
num_epochs= 50
accuracy2, accuracy_val2, loss2, loss_val2, acc_diz2 = training2(myModel2, df, num_epochs, batch__size)

In [ ]:
visualize_acc(accuracy2, accuracy_val2)

In [ ]:
visualize_loss(loss2, loss_val2)

In [ ]:
index_min = np.argmin(loss_val2)
index_min
plot_class_precision_and_recall(acc_diz2[index_min],accuracy_val2[index_min])

## Modello 3: CNN a 3 layer - Batch 32

In [ ]:
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import init

# ----------------------------
# Audio Classification Model
# ----------------------------
class AudioClassifier3 (nn.Module):
    # ----------------------------
    # Build the model architecture
    # ----------------------------
    def __init__(self):
        super().__init__()
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Xavier Initialization
        self.conv1 = nn.Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2,2)
        self.bn1 = nn.BatchNorm2d(8)
        init.xavier_normal_(self.conv1.weight)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1, self.pool1]

        # Second Convolution Block
        self.conv2 = nn.Conv2d(8, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2,2)
        self.bn2 = nn.BatchNorm2d(32)
        init.xavier_normal_(self.conv2.weight)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2, self.pool2]

        # Third Convolution Block
        self.conv3 = nn.Conv2d(32, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2,2)
        self.bn3 = nn.BatchNorm2d(128)
        init.xavier_normal_(self.conv3.weight)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        # Dropout 
        self.dropout = nn.Dropout(0.25)

        # Linear Classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=128, out_features=7)
        self.soft = nn.Softmax(dim=1)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)
 
    # ----------------------------
    # Forward pass computations
    # ----------------------------
    def forward(self, x):
        # Run the convolutional blocks
        x = self.conv(x)

        # Adaptive pool and flatten for input to linear layer
        x = self.ap(x)
        x = self.dropout(x)
        x = x.view(x.shape[0], -1)

        # Linear layer
        x = self.lin(x)
        x = self.soft(x)

        # Final output
        return x

# Create the model and put it on the GPU if available
myModel3 = AudioClassifier3()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel3 = myModel3.to(device)
# Check that it is on Cuda
next(myModel3.parameters()).device

In [ ]:
# ----------------------------
# Training Loop
# ----------------------------
def training3(model, dataset, num_epochs, batch__size):
  # Loss Function, Optimizer and Scheduler
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.01, weight_decay=0.005)
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01,
                                                steps_per_epoch=int(len(dataset)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')
  for layer in model.children():
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()

  # Random split of 2/3:1/3 between training and validation
  num_items = len(dataset)
  num_train = round(num_items * 2/3)
  num_val = num_items - num_train
  train_ds, val_ds, y_train, y_test = train_test_split(dataset.index, dataset['etichett-attore'],test_size=num_val, train_size=num_train,shuffle = True, random_state=123, stratify=dataset['etichett-attore'])


  # Create training and validation data loaders
  train_dl = torch.utils.data.DataLoader(train_ds, batch_size=int(batch__size/4), shuffle = True)
  val_dl = torch.utils.data.DataLoader(val_ds, batch_size=batch__size, shuffle = True)
  accuracy_train = []
  accuracy_val = []
  loss_train = []
  loss_validation = []
  acc_diz_list = []

  # early_stopping = 
  # Repeat for each epoch
  for epoch in range(num_epochs):
    running_loss = 0.0
    running_vloss = 0.0
    correct_prediction = 0
    total_prediction = 0

    # Repeat for each batch in the training set
    for i, data in enumerate(train_dl):

        train_spec = compute_spectrogram(dataset.iloc[data.tolist()])
        train_sound = SoundDS(train_spec)
        train_tensor = torch.utils.data.DataLoader(train_sound, batch_size=batch__size)
        for i in train_tensor:
          tt = i
        # Get the input features and target labels, and put them on the GPU
        inputs, labels = tt[0].to(device), tt[1].to(device)

        # Normalize the inputs
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s

        # Zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Keep stats for Loss and Accuracy
        running_loss += loss.item()

        # Get the predicted class with the highest score (posterior, predicted labels)
        _, prediction = torch.max(outputs,1)
    
        # Count of predictions that matched the target label
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]

      # Print stats at the end of the epoch
    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    
    acc = correct_prediction/total_prediction
    acc_val, loss_val, acc_diz = inference(dataset, model, criterion, val_dl, batch__size)
    acc_diz_list.append(acc_diz)
    accuracy_train.append(acc)
    accuracy_val.append(acc_val)
    loss_train.append(avg_loss)
    loss_validation.append(loss_val)

    print('Epoch:', epoch+1, 'Loss:', f'{avg_loss:.2f}', 'Accuracy:', acc)

  return accuracy_train, accuracy_val, loss_train, loss_validation, acc_diz_list

In [ ]:
seed = 245
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

batch__size = 32
num_epochs= 50
accuracy3, accuracy_val3, loss3, loss_val3, acc_diz3 = training3(myModel3, df, num_epochs, batch__size)

In [ ]:
visualize_acc(accuracy3, accuracy_val3)

In [ ]:
visualize_loss(loss3,loss_val3)

In [ ]:
index_min = np.argmin(loss_val3)
index_min
plot_class_precision_and_recall(acc_diz3[index_min],accuracy_val3[index_min])

##Modello 4: CNN a 4 layer - Batch 32

In [38]:
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import init

# ----------------------------
# Audio Classification Model
# ----------------------------
class AudioClassifier4 (nn.Module):
    # ----------------------------
    # Build the model architecture
    # ----------------------------
    def __init__(self):
        super().__init__()
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Xavier Initialization
        self.conv1 = nn.Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        self.pool1 = nn.MaxPool2d(2,2)
        init.xavier_normal_(self.conv1.weight)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1, self.pool1]

        # Second Convolution Block
        self.conv2 = nn.Conv2d(8, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2,2)
        init.xavier_normal_(self.conv2.weight)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2, self.pool2]

        # Third Convolution Block
        self.conv3 = nn.Conv2d(32, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2,2)
        init.xavier_normal_(self.conv3.weight)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3, self.pool3]

        # Fourth Convolution Block
        self.conv4 = nn.Conv2d(128, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(512)
        init.xavier_normal_(self.conv4.weight)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        # Dropout 
        self.dropout = nn.Dropout(0.25)

        # Linear Classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=512, out_features=7)
        self.soft = nn.Softmax(dim=1)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)
 
    # ----------------------------
    # Forward pass computations
    # ----------------------------
    def forward(self, x):
        # Run the convolutional blocks
        x = self.conv(x)

        # Adaptive pool and flatten for input to linear layer
        x = self.ap(x)
        x = self.dropout(x)
        x = x.view(x.shape[0], -1)


        # Linear layer
        x = self.lin(x)
        x = self.soft(x)

        # Final output
        return x

# Create the model and put it on the GPU if available
myModel4 = AudioClassifier4()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel4 = myModel4.to(device)
# Check that it is on Cuda
next(myModel4.parameters()).device

<IPython.core.display.Javascript object>

device(type='cpu')

In [39]:
# ----------------------------
# Training Loop
# ----------------------------
def training4(model, dataset, num_epochs, batch__size):
  # Loss Function, Optimizer and Scheduler
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.002, weight_decay=0.01)
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.002,
                                                steps_per_epoch=int(len(dataset)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')
  for layer in model.children():
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()

  # Random split of 2/3:1/3 between training and validation
  num_items = len(dataset)
  num_train = round(num_items * 2/3)
  num_val = num_items - num_train
  train_ds, val_ds, y_train, y_test = train_test_split(dataset.index, dataset['etichett-attore'],test_size=num_val, train_size=num_train,shuffle = True, random_state=123, stratify=dataset['etichett-attore'])


  # Create training and validation data loaders
  train_dl = torch.utils.data.DataLoader(train_ds, batch_size=int(batch__size/4), shuffle = True)
  val_dl = torch.utils.data.DataLoader(val_ds, batch_size=batch__size, shuffle = True)
  accuracy_train = []
  accuracy_val = []
  loss_train = []
  loss_validation = []
  acc_diz_list = []

  # Repeat for each epoch
  for epoch in range(num_epochs):
    running_loss = 0.0
    running_vloss = 0.0
    correct_prediction = 0
    total_prediction = 0

    # Repeat for each batch in the training set
    for i, data in enumerate(train_dl):

        train_spec = compute_spectrogram(dataset.iloc[data.tolist()])
        train_sound = SoundDS(train_spec)
        train_tensor = torch.utils.data.DataLoader(train_sound, batch_size=batch__size)
        for i in train_tensor:
          tt = i
        # Get the input features and target labels, and put them on the GPU
        inputs, labels = tt[0].to(device), tt[1].to(device)

        # Normalize the inputs
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s

        # Zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Keep stats for Loss and Accuracy
        running_loss += loss.item()

        # Get the predicted class with the highest score (posterior, predicted labels)
        _, prediction = torch.max(outputs,1)
    
        # Count of predictions that matched the target label
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]

      # Print stats at the end of the epoch
    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    
    acc = correct_prediction/total_prediction
    acc_val, loss_val, acc_diz = inference(dataset, model, criterion, val_dl, batch__size)
    
    accuracy_train.append(acc)
    accuracy_val.append(acc_val)
    loss_train.append(avg_loss)
    loss_validation.append(loss_val)
    acc_diz_list.append(acc_diz)

    print('Epoch:', epoch+1, 'Loss:', f'{avg_loss:.2f}', 'Accuracy:', acc)

  return accuracy_train, accuracy_val, loss_train, loss_validation, acc_diz_list

<IPython.core.display.Javascript object>

In [ ]:
seed = 245
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

batch__size = 32
num_epochs= 50
accuracy4, accuracy_val4, loss4, loss_val4, acc_diz4 = training4(myModel4, df, num_epochs, batch__size)

In [ ]:
visualize_acc(accuracy4, accuracy_val4)

In [ ]:
visualize_loss(loss4, loss_val4)

In [ ]:
index_min = np.argmin(loss_val4)
index_min
plot_class_precision_and_recall(acc_diz4[index_min],accuracy_val4[index_min])

##Modello finale: CNN a 3 layer - Batch 32 + Early stopping



In [ ]:
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import init

# ----------------------------
# Audio Classification Model
# ----------------------------
class AudioClassifier5 (nn.Module):
    # ----------------------------
    # Build the model architecture
    # ----------------------------
    def __init__(self):
        super().__init__()
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Xavier Initialization
        self.conv1 = nn.Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        self.pool1 = nn.MaxPool2d(2,2)
        init.xavier_normal_(self.conv1.weight)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1, self.pool1]

        # Second Convolution Block
        self.conv2 = nn.Conv2d(8, 32, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2,2)
        init.xavier_normal_(self.conv2.weight)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2, self.pool2]

        # Third Convolution Block
        self.conv3 = nn.Conv2d(32, 128, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(128)
        init.xavier_normal_(self.conv3.weight)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        # Dropout 
        self.dropout = nn.Dropout(0.25)

        # Linear Classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=128, out_features=7)
        self.soft = nn.Softmax(dim=1)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)
 
    # ----------------------------
    # Forward pass computations
    # ----------------------------
    def forward(self, x):
        # Run the convolutional blocks
        x = self.conv(x)

        # Adaptive pool and flatten for input to linear layer
        x = self.ap(x)
        x = self.dropout(x)
        x = x.view(x.shape[0], -1)

        # Linear layer
        x = self.lin(x)
        x = self.soft(x)

        # Final output
        return x

# Create the model and put it on the GPU if available
myModel5 = AudioClassifier5()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel5 = myModel5.to(device)
# Check that it is on Cuda
next(myModel5.parameters()).device

In [ ]:
# ----------------------------
# Training Loop
# ----------------------------
def training5(model, dataset, num_epochs, batch__size):
  # Loss Function, Optimizer and Scheduler
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.01, weight_decay=0.005)
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01,
                                                steps_per_epoch=int(len(dataset)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')
  for layer in model.children():
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()

  # Random split of 2/3:1/3 between training and validation
  num_items = len(dataset)
  num_train = round(num_items * 2/3)
  num_val = num_items - num_train
  train_ds, val_ds, y_train, y_test = train_test_split(dataset.index, dataset['etichett-attore'],test_size=num_val, train_size=num_train,shuffle = True, random_state=123, stratify=dataset['etichett-attore'])


  # Create training and validation data loaders
  train_dl = torch.utils.data.DataLoader(train_ds, batch_size=int(batch__size/4), shuffle = True)
  val_dl = torch.utils.data.DataLoader(val_ds, batch_size=batch__size, shuffle = True)
  accuracy_train = []
  accuracy_val = []
  loss_train = []
  loss_validation = []

  # Repeat for each epoch
  for epoch in range(num_epochs):
    running_loss = 0.0
    running_vloss = 0.0
    correct_prediction = 0
    total_prediction = 0

    # Repeat for each batch in the training set
    for i, data in enumerate(train_dl):

        train_spec = compute_spectrogram(dataset.iloc[data.tolist()])
        train_sound = SoundDS(train_spec)
        train_tensor = torch.utils.data.DataLoader(train_sound, batch_size=batch__size)
        for i in train_tensor:
          tt = i
        # Get the input features and target labels, and put them on the GPU
        inputs, labels = tt[0].to(device), tt[1].to(device)

        # Normalize the inputs
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s

        # Zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Keep stats for Loss and Accuracy
        running_loss += loss.item()

        # Get the predicted class with the highest score (posterior, predicted labels)
        _, prediction = torch.max(outputs,1)
    
        # Count of predictions that matched the target label
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]

      # Print stats at the end of the epoch
    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    
    acc = correct_prediction/total_prediction
    acc_val, loss_val, acc_diz = inference(dataset, model, criterion, val_dl, batch__size)
    accuracy_train.append(acc)
    accuracy_val.append(acc_val)
    loss_train.append(avg_loss)
    loss_validation.append(loss_val)

    if (epoch>10) and (all(ele > min(loss_validation) for ele in loss_validation[-4:])):
      break

    print('Epoch:', epoch+1, 'Loss:', f'{avg_loss:.2f}', 'Accuracy:', acc)

  return accuracy_train, accuracy_val, loss_train, loss_validation, acc_diz

In [ ]:
seed = 245
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

batch__size = 32
num_epochs= 50
accuracy5, accuracy_val5, loss5, loss_val5, acc_diz5 = training5(myModel5, df, num_epochs, batch__size)

In [ ]:
visualize_acc(accuracy5, accuracy_val5)

In [ ]:
visualize_loss(loss5, loss_val5)

In [ ]:
plot_class_precision_and_recall(acc_diz5,accuracy_val5[-1])

## Modello finale: CNN a 3 layer - Batch 32 + K-fold e Early Stopping (per verifica)

In [ ]:
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import init

# ----------------------------
# Audio Classification Model
# ----------------------------
class AudioClassifier6 (nn.Module):
    # ----------------------------
    # Build the model architecture
    # ----------------------------
    def __init__(self):
        super().__init__()
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Xavier Initialization
        self.conv1 = nn.Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2,2)
        self.bn1 = nn.BatchNorm2d(8)
        init.xavier_normal_(self.conv1.weight)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1, self.pool1]

        # Second Convolution Block
        self.conv2 = nn.Conv2d(8, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2,2)
        self.bn2 = nn.BatchNorm2d(32)
        init.xavier_normal_(self.conv2.weight)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2, self.pool2]

        # Third Convolution Block
        self.conv3 = nn.Conv2d(32, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2,2)
        self.bn3 = nn.BatchNorm2d(128)
        init.xavier_normal_(self.conv3.weight)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        # Dropout 
        self.dropout = nn.Dropout(0.25)

        # Linear Classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=128, out_features=7)
        self.soft = nn.Softmax(dim=1)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)
 
    # ----------------------------
    # Forward pass computations
    # ----------------------------
    def forward(self, x):
        # Run the convolutional blocks
        x = self.conv(x)

        # Adaptive pool and flatten for input to linear layer
        x = self.ap(x)
        x = self.dropout(x)
        x = x.view(x.shape[0], -1)

        # Linear layer
        x = self.lin(x)
        x = self.soft(x)

        # Final output
        return x

# Create the model and put it on the GPU if available
myModel6_1 = AudioClassifier6()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel6_1 = myModel6_1.to(device)
myModel6_2 = AudioClassifier6()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel6_2 = myModel6_2.to(device)
myModel6_3 = AudioClassifier6()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel6_3 = myModel6_3.to(device)
# Check that it is on Cuda
next(myModel6_1.parameters()).device
next(myModel6_2.parameters()).device
next(myModel6_3.parameters()).device

In [ ]:
# ----------------------------
# Training Loop
# ----------------------------
def training6(models, dataset, num_epochs, batch__size, splits):
  # Loss Function, Optimizer and Scheduler

  accuracy_train = []
  accuracy_val = []
  loss_train = []
  loss_validation = []
  splits.get_n_splits(dataset['audio'], dataset['etichetta'])

  for fold, (train_idx, val_idx) in enumerate(splits.split(dataset['audio'], dataset['etichett-attore'])):
    print('Fold {}'.format(fold + 1))
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(models[fold].parameters(),lr=0.01,weight_decay=0.005)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01,
                                                steps_per_epoch=int(len(dataset)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')
    
    for layer in models[fold].children():
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()
  
    train_dl = torch.utils.data.DataLoader(train_idx, batch_size=int(batch__size/4), shuffle = True)
    val_dl = torch.utils.data.DataLoader(val_idx, batch_size=batch__size, shuffle = True)
    accuracy_train_fold = []
    accuracy_val_fold = []
    loss_train_fold = []
    loss_val_fold = []

    # Repeat for each epoch
    for epoch in range(num_epochs):
      running_loss = 0.0
      running_vloss = 0.0
      correct_prediction = 0
      total_prediction = 0

      # Repeat for each batch in the training set
      for i, data in enumerate(train_dl):
          train_spec = compute_spectrogram(dataset.iloc[data.tolist()])
          train_sound = SoundDS(train_spec)
          train_tensor = torch.utils.data.DataLoader(train_sound, batch_size=batch__size)
          for i in train_tensor:
            tt = i
          # Get the input features and target labels, and put them on the GPU
          inputs, labels = tt[0].to(device), tt[1].to(device)

          # Normalize the inputs
          inputs_m, inputs_s = inputs.mean(), inputs.std()
          inputs = (inputs - inputs_m) / inputs_s

          # Zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = models[fold](inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          scheduler.step()

          # Keep stats for Loss and Accuracy
          running_loss += loss.item()

          # Get the predicted class with the highest score (posterior, predicted labels)
          _, prediction = torch.max(outputs,1)
    
          # Count of predictions that matched the target label
          correct_prediction += (prediction == labels).sum().item()
          total_prediction += prediction.shape[0]
          if epoch == (num_epochs-1):
            for label_index,label in enumerate(labels):
              acc_diz[int(label)][0] += 1 
              acc_diz[int(label)][1] += int(label == prediction[label_index])
      
      # Print stats at the end of the epoch
      num_batches = len(train_dl)
      avg_loss = running_loss / num_batches
      
      acc = correct_prediction/total_prediction
      acc_val, loss_val, acc_diz = inference(dataset, models[fold], criterion, val_dl, batch__size)

      accuracy_val_fold.append(acc_val)
      accuracy_train_fold.append(acc)
      loss_train_fold.append(avg_loss)
      loss_val_fold.append(loss_val)
      
      if all(ele > min(loss_val_fold) for ele in loss_val_fold[-4:]):
        break

      print('Epoch:', (epoch+1), 'Loss:', f'{avg_loss:.2f}', 'Accuracy:', acc)
    
    accuracy_train.append(accuracy_train_fold)
    accuracy_val.append(accuracy_val_fold)
    loss_train.append(loss_train_fold)
    loss_validation.append(loss_val_fold)

  return accuracy_train, accuracy_val, loss_train, loss_validation, acc_diz 

In [ ]:
seed = 245
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

splits = StratifiedKFold(n_splits=3, shuffle=True, random_state=123)
batch__size = 32
num_epochs= 50
accuracy6, accuracy_val6, loss6, loss_val6, acc_diz6 = training6([myModel6_1,myModel6_2,myModel6_3], df, num_epochs, batch__size, splits)

In [ ]:
visualize_acc(accuracy6, accuracy_val6)

In [ ]:
visualize_loss(loss6, loss_val6)